In [35]:
import os
os.system('python test.py --trained_model="craft_mlt_25k.pth" --test_folder="/home/yukinori/Desktop/segmentation_model/self_supervised_cropped" --text_threshold=0.8 --result_folder="txt_result/"')

/home/yukinori/miniconda3/envs/pytorch_bootcamp/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/yukinori/miniconda3/envs/pytorch_bootcamp/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Loading weights from checkpoint (craft_mlt_25k.pth)
elapsed time : 214.97649574279785s/Desktop/segmentation_model/self_supervised_cropped/Nobleza_virtuosa_53_left.jpggg


0

In [16]:
import cv2
import numpy as np
import os

def save_cropped_images(image_dir, txt_dir, output_dir):
	if not os.path.exists(output_dir):
		os.makedirs(output_dir)
	txt_list = os.listdir(txt_dir)
	for txt_file in txt_list:
		img = cv2.imread(os.path.join(image_dir, txt_file.replace(".txt", ".jpg")))
		with open(os.path.join(txt_dir, txt_file), "r") as f:
			lines = f.readlines()
			for i, line in enumerate(lines):
				processed_line = line.strip('\n')
				processed_line = processed_line.split(',')
				pts = np.array(processed_line, np.float32)
				pts = pts.reshape((-1, 2))
				width = int(max(np.linalg.norm(pts[1] - pts[0]), np.linalg.norm(pts[3] - pts[2])))
				height = int(max(np.linalg.norm(pts[2] - pts[1]), np.linalg.norm(pts[3] - pts[0])))
				dst = np.array([
					[0, 0],
					[width - 1, 0],
					[width - 1, height - 1],
					[0, height - 1]
				], dtype="float32")
				M = cv2.getPerspectiveTransform(pts, dst)
				warped = cv2.warpPerspective(img, M, (width, height))
				cv2.imwrite(os.path.join(output_dir, f"{i}_{txt_file.replace('.txt', '.jpg')}"), warped)

In [17]:
import cv2
import numpy as np
import os
import csv

def save_line_bbox(img_dir, txt_dir, output_dir, threshold=20):
    assert output_dir != txt_dir
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    txt_list = os.listdir(txt_dir)
    for txt_file in txt_list:
        img = cv2.imread(os.path.join(img_dir, txt_file.replace(".txt", ".jpg")))
        height, width = img.shape[:2]
        with open(os.path.join(txt_dir, txt_file), "r") as f:
            reader = csv.reader(f)
            bboxes = list(reader)
            centers = [(int(bbox[1]) + int(bbox[5])) // 2 for bbox in bboxes]
            bboxes = [x for _, x in sorted(zip(centers, bboxes))]
            if bboxes == []:
                continue
            lines = []
            line = [bboxes[0]]
            line_first_center = 0
            for i in range(1, len(bboxes)):
                if abs(centers[i] - centers[line_first_center]) < threshold:
                    line.append(bboxes[i])
                else:
                    line_first_center = i
                    lines.append(line)
                    line = [bboxes[i]]
            lines.append(line)
        for i, line in enumerate(lines):
            # min_x = min_y = int('inf')
            min_x = width
            min_y = height
            max_x = max_y = 0
            for bbox in line:
                min_x = min(min_x, int(bbox[0]), int(bbox[6]))
                min_y = min(min_y, int(bbox[1]), int(bbox[3]))
                max_x = max(max_x, int(bbox[2]), int(bbox[4]))
                max_y = max(max_y, int(bbox[5]), int(bbox[7]))
            with open(os.path.join(output_dir, f"{txt_file}"), "a") as f:
                f.write(f"{min_x},{min_y},{max_x},{min_y},{max_x},{max_y},{min_x},{max_y}\n")

In [18]:
save_line_bbox("/home/yukinori/Desktop/segmentation_model/self_supervised_cropped", "txt_result", "line_txt_result")

In [20]:
save_cropped_images("/home/yukinori/Desktop/segmentation_model/self_supervised_cropped_colored", "line_txt_result", "cropped_images_line")

In [3]:
import cv2
test_img = cv2.imread("cropped_images_v2/183_Noble_perfecto_5_left.jpg")

In [4]:
test_img.shape

(34, 119, 3)